In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [93]:
interactions_test = pd.read_csv('data/interactions_test.csv')
interactions_train = pd.read_csv('data/interactions_train.csv')
n_train = len(interactions_train)
n_test = len(interactions_test)
#RAW_interactions = pd.read_csv('data/RAW_interactions.csv')
RAW_recipes = pd.read_csv('data/RAW_recipes.csv')
interactions_train.shape

(698901, 6)

In [94]:
# RAW_recipes[RAW_recipes["id"].isin([2496, 32844, 46877])]

In [95]:
print(f"Nombre d'utilisateurs: {len(interactions_train.user_id.unique())}")
print(f"Nombre de recettes: {len(interactions_train.recipe_id.unique())}")

Nombre d'utilisateurs: 25076
Nombre de recettes: 160901


In [96]:
interactions_train.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [97]:
# filter recipe
interactions_train['count'] = interactions_train.groupby('recipe_id').transform('count')['user_id']
movieId = interactions_train.drop_duplicates('recipe_id').sort_values(
    'count', ascending=False).iloc[:100]['recipe_id']
interactions_train = interactions_train[interactions_train['recipe_id'].isin(movieId)].reset_index(drop=True)

#filter users
interactions_train['count'] = interactions_train.groupby('user_id').transform('count')['recipe_id']
user_id = interactions_train.drop_duplicates('user_id').sort_values(
    'count', ascending=False).iloc[:20001]['user_id']
interactions_train = interactions_train[interactions_train['user_id'].isin(user_id)].reset_index(drop=True)

interactions_train.shape

(35843, 7)

In [98]:
print(f"Nombre d'utilisateurs: {len(interactions_train.user_id.unique())}")
print(f"Nombre de recettes: {len(interactions_train.recipe_id.unique())}")

Nombre d'utilisateurs: 12725
Nombre de recettes: 100


In [99]:
pivot_train = interactions_train.pivot_table(index = ["user_id"],columns = ["recipe_id"],values = "rating")#, fill_value=0)
#pivot_test = interactions_test.pivot_table(index = ["user_id"],columns = ["recipe_id"],values = "rating")
pivot_train.head(10)

recipe_id,2496,2886,3470,5170,8701,8782,9272,9836,10744,15072,...,121490,128956,129345,129926,135350,141983,150384,150863,200296,221743
user_id,,,,,,,,,,,,,,,,,,,,,
1533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1535,NaN,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1792,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1891,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2310,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2312,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2586,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
recipe_vector = pivot_train[2496]
similarity = pivot_train.corrwith(recipe_vector) 
similarity = similarity.sort_values(ascending=False)
similarity.head()

recipe_id
2496      1.000000
32844     0.938315
46877     0.909826
71373     0.889824
141983    0.802955
dtype: float64

## NCF

In [ ]:
class NCF(nn.Module):
        
    def __init__(self, n_users, n_items, n_factors=8):
        super().__init__()
        self.user_embeddings = torch.nn.Embedding(n_users, n_factors)
        self.item_embeddings = torch.nn.Embedding(n_items, n_factors)
        self.predictor = torch.nn.Sequential(
            nn.Linear(in_features=n_factors*2, out_features=64),
            nn.Linear(in_features=64, out_features=32),
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )
        
        
    def forward(self, user, item):
        

        u = self.user_embeddings(user)
        i = self.item_embeddings(item)

        # Concat the two embedding layers
        z = torch.cat([u, i], dim=-1)
        return self.predictor(z)

In [ ]:
model = NCF(n_user, n_items).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
train(model, optimizer, trainloader, epochs=5)

In [ ]:
test(model, testloader)

In [ ]:
users, movies, r = next(iter(testloader))
users = users.cuda()
movies = movies.cuda()
r = r.cuda()

y = model(users, movies)*5
print("interactions_train", r[:10].data)
print("predictions:", y.flatten()[:10].data)